In [1]:
# Import Statements 
####################
import os 
import sys
import copy
import pandas as pd
import pathlib as pl
import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image, ImageDraw
import pathlib
import datetime
import itertools as ite 
import math
import calendar

import shapefile  #conda install -c conda-forge pyshp    # (version should be 2.0)
from shapely.geometry import Point   #conda install -c conda-forge shapely
from shapely.geometry import shape


NUM_TIME_BINS_PER_DAY = 24

In [3]:
_="""

Define any useful functions 

"""

def cleanURL(url):
    p = pathlib.Path(url)
    path = str(p.as_posix()) 
    return path 


def getDF(loc, sheetname):
    dataframe = pd.read_excel(loc, sheetname)
    #https://stackoverflow.com/questions/40950310/strip-trim-all-strings-of-a-dataframe
    dataframe = dataframe.applymap(lambda x: x.strip() if type(x) is str else x)
    return dataframe

def printNulls(df):
    null_columns = df.columns[df.isnull().any()]
    return df[null_columns].isnull().sum() 


def writeDFToFile(dfs, path_): #dfs is an array of dataframes and their sheet names , path needs to have
    time_ = str(datetime.datetime.now())
    current_date_time = time_[0:time_.index(".")]
    current_date_time = current_date_time.replace(":", "-")
    task4_fileoutput = path_+current_date_time+".xlsx"

    writer = pd.ExcelWriter(task4_fileoutput)
    
    for df_tuple in dfs:  
        df = df_tuple[0]
        sheetName = df_tuple[1]
        df.to_excel(writer, sheetName)
    print("file written to :       " + task4_fileoutput)
    writer.save()



In [6]:
#Save final_dataframe for reference
#final_dataframe.to_csv(cleanURL(r'C:\Users\j70514\Documents\Data Science Stuff\DeepLearning_cs230\CNN_data_crunch/buildOff3.csv') , sep = ',' )


# Really, after we finalize input dataset
# retrieve final_dataframe and start from here
final_dataframe =pd.read_csv(cleanURL(r'C:\Users\User\Documents\CS230 Project\new_github\final_dataframe_after_removing_extreme_vals.csv'))
final_dataframe['Date'] = pd.to_datetime(final_dataframe['Date'], format ='%Y-%m-%d %H:%M:%S')
print('done')
# assert(final_dataframe.Date)

done


In [7]:
final_dataframe.head()


,Unnamed: 0,Unnamed: 0.1,ID,Date,Primary Type,Beat,District,Ward,Community Area,Latitude,Longitude,YEAR,MONTH,DAY,WEEKDAY,TIME_OF_DAY,PRECIPITATION,MAX TEMP,MIN TEMP
0,0,0,10000092,2015-03-18 07:44:00,BATTERY,1111,11,28,25,41.891399,-87.744385,2015,3,18,Wednesday,7,0.0,49.0,24.0
1,1,1,10000094,2015-03-18 11:00:00,OTHER OFFENSE,725,7,15,67,41.773372,-87.665319,2015,3,18,Wednesday,11,0.0,49.0,24.0
2,2,2,10000095,2015-03-18 10:45:00,BATTERY,222,2,4,39,41.813861,-87.596643,2015,3,18,Wednesday,10,0.0,49.0,24.0
3,3,3,10000096,2015-03-18 10:30:00,BATTERY,225,2,3,40,41.800802,-87.622619,2015,3,18,Wednesday,10,0.0,49.0,24.0
4,4,4,10000097,2015-03-18 09:00:00,ROBBERY,1113,11,28,25,41.878065,-87.743354,2015,3,18,Wednesday,9,0.0,49.0,24.0


In [52]:
point_ = (final_dataframe.iat[0,9], final_dataframe.iat[0,8])
point_

(-87.74438456700001, 41.891398861)

# GIS Shape Files Functions to convert Map data to CNN data (library)

In [8]:
# sf.shapes()[0].__geo_interface__
# feature = sf.shapeRecords()[0]
# feature.record
# feature.shape.shapeTypeName

##check if all the shapes are valid polygons in a shapefile 
def checkIfShapeFileFilledWithPolygons(sfile):
    for shape_ in sfile.shapes():
        assert(shape_.shapeType == 5) # the shape.shapeTypeName is 'POLYGON'


#https://gis.stackexchange.com/questions/250172/finding-out-if-coordinate-is-within-shapefile-shp-using-pyshp
def return_Category_A_Point_Belongs_In(colIndex, sfile, point):  # (longitude,latitude) wise 
    features = sf.shapeRecords()
    record_ = None
    shape_boundary = None
    for i, feature in enumerate(features):
        record_ = feature.record
        shape_boundary = feature.shape
        if(Point(point).within(shape(shape_boundary))):
            return record_[colIndex], shape_boundary
    #if none found, it equals none 
    return None, None

def getShapeThatCorrespondsToCategory(colIndex, category, sfile):
    records = sfile.records()
    for i, record in enumerate(records):
#         print(i)
        if(record[colIndex] == category):
            return sfile.shapes()[i]
        
    #if none found, it equals none 
    return None


#optimized for 30x speed up over other functions 
#list_of_categories must be list of ints
#colIndex must be an integer
def maskGISPolygonsData(colIndex, list_of_categories, iHeight, iWidth,sf ): # sf is shapes file 
    iHeight = int(iHeight)
    iWidth = int(iWidth)

    records = sf.records()
    actualCategoriesFoundInShapeFile = []
    for record in records:
        category = int(record[colIndex])
        actualCategoriesFoundInShapeFile.append(category)

    shapesOfShapeFile = sf.shapes()
    # now that we have the categories the data provides, we loop through the categories the data has
    # if there are categories that the data has that shape file does not => leave None in that list 
    shapes= []
    #shapes = sf.shapes()
    list_ = list_of_categories
    lookup_ = {}

    list_ = list_of_categories
    locationInShapeFile = None 
    #look up dictionary 
    for i, ele in enumerate(list_):
        if(ele in actualCategoriesFoundInShapeFile):
            locationInShapeFile = actualCategoriesFoundInShapeFile.index(ele)
            shapes.append(shapesOfShapeFile[locationInShapeFile])
        else: 
            shapes.append(None) 
        lookup_[i+1] = ele #we are saying that this category resides in i+1 in the Masks table  

    assert(len(shapes) == len(lookup_))
    #set the masks
    MASKS= np.zeros((len(lookup_)+1, iHeight, iWidth)) # we add a category layer and hence we added the +1 -> this allows us to have a -1 category 

    ## look up dictionary for points 
    pointsLU = {} # points look up 

    for row in range(iHeight):
        for col in range(iWidth):

            colMid = col+.5 # make the latitude and longitude be in the middle of the pixel
            rowMid = iHeight - row - 1 +.5 # double check for this to work 
            #rowMid = row +.5
            
            longVal = (colMid-0.)*(1./longMultiplier)+longMin2
            latVal = (rowMid -0.)*(1./latMultiplier)+latMin2
            #point_ = (longVal,latVal)

            #plug in latVal and longVal
            pointsLU[(row, col)] = Point((longVal, latVal))


    #now for each category, apply the row col to get the mask 

    for ishape, shape_ in enumerate(shapes):
        if(type(shape_) == type(None)): #we didn't find a shape for the element at this index
            print("found no shape")
            continue # no shape -> this category cannot compete in the code 
        boundary = shape(shape_)
        for row in range(iHeight):
            for col in range(iWidth):
                if(len(np.where(MASKS[:,row,col] ==1)[0]) == 1):
                    continue
            
                #now check if point exists within boundary 
                if(pointsLU[(row, col)].within(boundary)):  
                    MASKS[ishape+1, row, col] = 1.   # shift by +1 because at 0 we will have -1 layer
    
    for row in range(iHeight):
        for col in range(iWidth):
            if(len(np.where(MASKS[:,row,col] ==1)[0]) == 0): # if nothing found for this row, col we put it in the null category 
                MASKS[0, row, col] = 1.
        
    
    return lookup_ , MASKS 

def convertImageToCategoryMask(colIndex, categoryVal, sfile ,iHeight , iWidth ,latMultiplier, longMultiplier, latMin2,longMin2 ):
    mask = np.zeros(shape=(iHeight, iWidth))
    shape_that_points_should_be_in = getShapeThatCorrespondsToCategory(colIndex, categoryVal, sfile)   
    
    #if None is returned, the shapefile didn't have that category and therefore this category has no mask, no point belongs
    if(type(shape_that_points_should_be_in) == type(None)):
        return mask # KLUDGE: should we have a mask of -1's since this category is not available?  or -1 when point belongs to no category
    
    boundary = shape(shape_that_points_should_be_in)
    
    for row in range(iHeight):
        for col in range(iWidth):
            colMid = col+.5 # make the latitude and longitude be in the middle of the pixel
            #rowMid = iHeight - row - 1 +.5 # double check for this to work 
            rowMid = row +.5
            
            longVal = (colMid-0.)*(1./longMultiplier)+longMin2
            latVal = (rowMid -0.)*(1./latMultiplier)+latMin2
            point_ = (longVal,latVal)
            
            if(Point(point_).within(boundary)):
                mask[row][col]= 1 
            
    return mask 

def convertImageToCategoryMask2(colIndex,  sfile ,iHeight  , iWidth ,latMultiplier, longMultiplier, latMin2,longMin2 ):
    iHeight = int(iHeight)
    iWidth = int(iWidth)
    mask = np.zeros(shape=(iHeight, iWidth))
#     shape_that_points_should_be_in = getShapeThatCorrespondsToCategory(colIndex, categoryVal, sfile)   
    
#     #if None is returned, the shapefile didn't have that category and therefore this category has no mask, no point belongs
#     if(type(shape_that_points_should_be_in) == type(None)):
#         return mask # KLUDGE: should we have a mask of -1's since this category is not available?  or -1 when point belongs to no category
    
#     boundary = shape(shape_that_points_should_be_in)
    
    for row in range(iHeight):
        for col in range(iWidth):
            colMid = col+.5 # make the latitude and longitude be in the middle of the pixel
            #rowMid = iHeight - row - 1 +.5 # double check for this to work 
            rowMid = row +.5
            
            longVal = (colMid-0.)*(1./longMultiplier)+longMin2
            latVal = (rowMid -0.)*(1./latMultiplier)+latMin2
            point_ = (longVal,latVal)
            
            r,s = return_Category_A_Point_Belongs_In(colIndex, sfile, point_)
            
            if(type(r) ==type(None)):
                r = -1
                
            r = int(r)
            mask[row][col]= r                
            
    return mask 

# # Test 1
# r,s = return_Category_A_Point_Belongs_In(2, sf, point_) # for community areas -> the 2nd column gives the comm area
# print(r)
# print(s)
# ############works!!!  returns category 25 which is the right value for community area for that point!!!!! 


# # Test 2
# sampleRecord = sf.records()[14]
# realShape = sf.shapes()[14]
# colWanted = sampleRecord[2] # for Community area -> look at column 2 
# shape_ = getShapeThatCorrespondsToCategory(2, colWanted, sf)

# # shape_.equals(realShape)
# # print(realShape.__geo_interface__['coordinates'])
# # print(shape_.__geo_interface__['coordinates'])
# a =shape_.__geo_interface__['coordinates']
# b =realShape.__geo_interface__['coordinates']
# print(a ==b ) #should be true



In [60]:
#Sample Usage

bc = maskGISPolygonsData(2, [j for j in range(1,78)], iHeight, iWidth,commAreaSF)
lookup , mask = bc
mask.shape  # remember there is an extra layer on the first element because we add the -1 class 
lookup
img = np.array(mask)
#to transform the image
for ind in range(1, len(img)):
    img[ind] = img[ind]*lookup[ind]
img[0] = img[0]*-1
resImage = np.sum(img, axis = 0)
np.save(cleanURL(r'C:\Users\User\Documents\CS230 Project\new_github\crime_prediction\Ali\GIS datasets/PDistrict_SHORT_WAY_maps256by256.npy'),resImage)
print('done')

# This is where we start the CNN image dataset generation 


In [54]:
############
# Step 1.Assume Latitude and Longitude are normally distributed, filter out anything +/- 2 sigmas for 95% of data - empirically  not many datapoints are removed
############  
#get the old values 
latMax1 = final_dataframe.Latitude.max()
latMin1 = final_dataframe.Latitude.min()
longMax1  = final_dataframe.Longitude.max()
longMin1  = final_dataframe.Longitude.min()
print(latMax1 - latMin1)
print(longMax1 - longMin1)

#get mean and standard deviations to use for filteration
latMean = final_dataframe.Latitude.mean()
longMean = final_dataframe.Longitude.mean()
latStd = final_dataframe.Latitude.std()  #we could toggle degrees of freedom but for now I don't think it matters 
longStd = final_dataframe.Longitude.std()

# perform filteration 
final_dataframe = final_dataframe.loc[(final_dataframe['Latitude'] < latMean+2*latStd) & (final_dataframe['Latitude'] > latMean-2*latStd)].copy()
final_dataframe = final_dataframe.loc[(final_dataframe['Longitude'] < longMean+2*longStd) & (final_dataframe['Longitude'] > longMean-2*longStd)].copy()





0.3779438509999977
0.4032663549999995
0.34226877799999755
0.23889396600000623


In [9]:
latMax2 = final_dataframe.Latitude.max()
latMin2 = final_dataframe.Latitude.min()
longMax2  = final_dataframe.Longitude.max()
longMin2  = final_dataframe.Longitude.min()
print(latMax2 - latMin2)
print(longMax2 - longMin2)

0.3565254240000044
0.24557258000000104


In [10]:
############
# Step 2. Generate Image Size and Scaling Multipliers to convert to image format. Have a 256 by 256 datasize. 
############

iHeight = 256.
iWidth = 256.

# pixel/degree ratios
latMultiplier = (iHeight- 0.)/(latMax2 - latMin2)  
longMultiplier = (iWidth- 0.)/(longMax2 - longMin2)

#convert the latitude and longitude values to the pixel values
final_dataframe['latPixel'] = (final_dataframe.Latitude - latMin2)*latMultiplier
final_dataframe['longPixel'] =(final_dataframe.Longitude - longMin2)*longMultiplier

#pixel values are floats and not integers. type cast
final_dataframe['latPixel'] = final_dataframe['latPixel'].astype(np.int64)
final_dataframe['longPixel'] = final_dataframe['longPixel'].astype(np.int64)

final_dataframe['latPixel'] = iHeight - final_dataframe['latPixel'] - 1  # this is important because matrix row numbers
#don't go from bottom to top but latitudes do. So we simply invert it. 
print("done")

done


In [11]:
final_dataframe.columns.tolist()
# final_dataframe['Date']
final_dataframe.Date.min()

Timestamp('2001-01-01 01:00:00')

In [12]:
###########  Provide masks for precincts, beats, district, and wards
# Step 3.  Get the data masks ready for any location vectors . We have 4 -> presinct, beat, community area
###########
COMM_AREAS = final_dataframe['Community Area'].unique().tolist() #77#[] #77
commAreaSF = shapefile.Reader(cleanURL(r'C:\Users\User\Documents\CS230 Project\new_github\crime_prediction\Ali\GIS datasets\geo_export_eb09b63d-cc4a-4268-8df4-e40c4f11a38b.shp'))


BEATS_AREAS = final_dataframe['Beat'].unique().tolist() 
beatsAreaSF = shapefile.Reader(cleanURL(r'C:\Users\User\Documents\CS230 Project\new_github\crime_prediction\Ali\GIS datasets\police beats\geo_export_d3113d24-90eb-442c-83d9-4208a84c7275.shp'))

DISTRICT_AREAS = final_dataframe['District'].unique().tolist()
districtsAreaSF = shapefile.Reader(cleanURL(r'C:\Users\User\Documents\CS230 Project\new_github\crime_prediction\Ali\GIS datasets\police districts\geo_export_2f172cc2-d492-418f-a4e6-a19358ac79bc.shp'))

WARD_AREAS = final_dataframe['Ward'].unique().tolist()
wardsAreaSF = shapefile.Reader(cleanURL(r'C:\Users\User\Documents\CS230 Project\new_github\crime_prediction\Ali\GIS datasets\wards\geo_export_ddb6bf98-2328-402f-b3f0-faf9cded16a8.shp'))

# PRECINCT_AREAS = final_dataframe['Ward'].unique().tolist()
# precinctsAreaSF = shapefile.Reader(cleanURL(r'C:\Users\User\Documents\CS230 Project\new_github\crime_prediction\Ali\GIS datasets\precincts\geo_export_16916c2a-daf5-4f30-97ee-18f919748e95.shp'))


# community_area_mask = np.zeros(COMM_AREAS, iHeight, iWidth)
# for j in range(NUM_COMMUNITY_AREAS):
#     community_area_mask[j] = convertImageToCategoryMask(2, str(j) , commAreaSF ,iHeight , iWidth ,latMultiplier, longMultiplier, latMin2,longMin2 )
    

In [13]:
def recordsPrint(sf):
    print(sf.fields)
    for record in sf.records():
        print(record[:])
        
def saveMaskImg(mask, lookup, path):
    img = np.array(mask)
    #to transform the image
    for ind in range(1, len(img)):
        img[ind] = img[ind]*lookup[ind]
    img[0] = img[0]*-1
    resImage = np.sum(img, axis = 0)
    np.save(cleanURL(path),resImage)

def saveMaskImg(mask, lookup, path, path2):
    keys =[]
    values=[]
    for key in lookup:
        keys.append(key)
        values.append(lookup[key])

    lookupDF = pd.DataFrame({'key':keys, 'value':values})
    lookupDF.to_csv(cleanURL(path2), sep = ',')
    np.save(cleanURL(path),mask)
    
        
# recordsPrint(commAreaSF)
# print(COMM_AREAS)
        
# recordsPrint(beatsAreaSF)
# print(BEATS_AREAS)

# recordsPrint(districtsAreaSF)
# print(DISTRICT_AREAS)

# recordsPrint(wardsAreaSF)
# print(WARD_AREAS)



In [14]:
base_path = r"C:\Users\User\Documents\CS230 Project\new_github\crime_prediction\Ali\GIS datasets\resultMasks\ "
base_path = base_path.strip()
base_path

'C:\\Users\\User\\Documents\\CS230 Project\\new_github\\crime_prediction\\Ali\\GIS datasets\\resultMasks\\'

In [15]:
#comm area
COMM_AREA_MASK_LOOKUP , COMM_AREA_MASK = maskGISPolygonsData(2, COMM_AREAS, iHeight, iWidth,commAreaSF)
saveMaskImg(COMM_AREA_MASK, COMM_AREA_MASK_LOOKUP, path=base_path+'commArea.npy', path2=base_path+'commAreaL.csv')
print('done')

found no shape
done


In [16]:
#BEATS 
BEATS_AREA_MASK_LOOKUP , BEATS_AREA_MASK = maskGISPolygonsData(1, BEATS_AREAS, iHeight, iWidth,beatsAreaSF)
saveMaskImg(BEATS_AREA_MASK, BEATS_AREA_MASK_LOOKUP, path=base_path+'beatArea.npy', path2=base_path+'beatAreaL.csv')
print('done')#5 beats not found

found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
found no shape
done


In [17]:
#Districts
DISTRICT_AREA_MASK_LOOKUP , DISTRICT_AREA_MASK = maskGISPolygonsData(1, DISTRICT_AREAS, iHeight, iWidth,districtsAreaSF)
saveMaskImg(DISTRICT_AREA_MASK, DISTRICT_AREA_MASK_LOOKUP, path=base_path+'districtArea.npy', path2=base_path+'districtAreaL.csv')
print('done')

found no shape
done


In [18]:
# WARDS 
WARD_AREA_MASK_LOOKUP , WARD_AREA_MASK = maskGISPolygonsData(2, WARD_AREAS, iHeight, iWidth,wardsAreaSF)
saveMaskImg(WARD_AREA_MASK, WARD_AREA_MASK_LOOKUP, path=base_path+'wardArea.npy', path2=base_path+'wardAreaL.csv')
print('done')

done


In [19]:
#PRECINCTS 
# not in dataset
base_path

'C:\\Users\\User\\Documents\\CS230 Project\\new_github\\crime_prediction\\Ali\\GIS datasets\\resultMasks\\'

In [72]:
###########
# Step 4. Get ready for the Data Generation Loop 
###########


#initialize values for the for loop 
mindate = final_dataframe.Date.min().date()
maxdate = final_dataframe.Date.max().date()

delta = maxdate - mindate

#https://stackoverflow.com/questions/7274267/print-all-day-dates-between-two-dates
datesInDays = [ mindate + datetime.timedelta(index) for index in range(delta.days+1)] 
timeOfDays = [i for i in range(NUM_TIME_BINS_PER_DAY)]


#we want the cartesian product of days and the time of days -> this is how many images we are making
dates_and_timeOfDays_iterator = ite.product(datesInDays, timeOfDays)
dates_and_timeOfDays = [[z[0], z[1]] for z in dates_and_timeOfDays_iterator]  # has the day and the time of day

#check we got all possible combinations 
assert(len(dates_and_timeOfDays) == NUM_TIME_BINS_PER_DAY * len(datesInDays))

print(len(dates_and_timeOfDays))
#https://stackoverflow.com/questions/13635032/what-is-the-inverse-function-of-zip-in-python
#allDates, allTimesOfDay  = zip(*dates_and_timeOfDays) # we unzip the cartesian product so that we can now loop through everything

148560


In [ ]:
###########
# Step 5. The For loop for generating the images 
###########


#The General Algorithm:
# For each day:
#   For each time of day:
#       dataframe = get the data points for this time period
#       generate an image


# For faster filtering, remove some columns
all_cols = final_dataframe.columns.tolist()
all_cols.remove('ID')
all_cols.remove('Latitude') # have the pixelated values already
all_cols.remove('Longitude')
fDF = final_dataframe[all_cols].copy()

#assert datatypes
assert(final_dataframe['TIME_OF_DAY'].dtype == np.dtype('int32'))
assert(final_dataframe['DAY'].dtype == np.dtype('int64'))
assert(final_dataframe['MONTH'].dtype == np.dtype('int64'))
assert(final_dataframe['YEAR'].dtype == np.dtype('int64'))


LAYERS_IN_A_IMAGE = 5

images_x = np.zeros(shape = (len(dates_and_timeOfDays), LAYERS_IN_A_IMAGE , int(iHeight),int(iWidth)))
outputs_y = np.zeros(10) 

outputs = []

dt = None
tofD = None 
zero_img = np.zeros(shape = (int(iHeight),int(iWidth)))



In [ ]:
## Step 5 continued... the actual for loop for adding data

for i, _ in enumerate(dates_and_timeOfDays):
    dt = _[0]
    tofD = _[1]
    df = fDF.loc[(tofD == fDF['TIME_OF_DAY']) & (dt.day == fDF['DAY']) & (dt.month == fDF['MONTH']) & (dt.year == fDF['YEAR'])]

    img = np.zeros((LAYERS_IN_A_IMAGE,zero_img.shape[0], zero_img.shape[1]))
    #the lat/long base layer
    #img[0] = np.array(zero_img)
    if(len(df) != 0):
        row = df.latPixel.values
        col = df.longPixel.values
        img[0][row,col] = 1.
        #to check:  a = np.where(img ==1), a[0] , a[1]
    
    #temperature layers 
    # Max temperature 
    img[1] = df['MAX TEMP'].iat[0]
    # Min temperature 
    img[2] =  df['MIN TEMP'].iat[0]
    # PRECIPITATION
    img[3] =  df['PRECIPITATION'].iat[0]
    
    # TODO: add more layers here

    images_x[i] = img 

In [ ]:


#######
# Save data

np.save(r'/Volumes/GoogleDrive/My Drive/Crime Data/Composite Data/Sean Workspace/27_November_cnn.npy', images_x)
np.save(r'/Volumes/GoogleDrive/My Drive/Crime Data/Composite Data/Sean Workspace/27_November_yOutput.npy', outputs_y)



